In [1]:
import pdb
import torch, os
import argparse, datetime
import gc,time
from tqdm import tqdm
import __init__


from src.gp_cpab.src.transformation.configManager import configManager
from src.gp_cpab.src.extra.experiment_utilities import *

from src.gp_cpab.src.transformation.gp_cpab import gp_cpab,gp_interpolation
from src.gp_cpab.src.transformation.configManager import configManager

import torch

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [2]:
import argparse; import sys; sys.argv=['']; del sys
import math

def define_prior_init(channels, space='log'):
    prior_init = []
    base_alpha = channels-2

    if space=='log':
        #prior_init = [math.log(1e-6)]*channels
        prior_init = [math.log(1e-6), *([ math.log(1/base_alpha) ]*base_alpha)]
    else:
        # Current Approach
        #prior_init = [0.0, *([1/base_alpha]*base_alpha)]

        # Approach A for avoiding flat landscape in the optimization - avoiding local optima
        prior_init = [1.0, 0.0, *([0.0]*base_alpha) ]

        # Approach B for avoiding flat landscape in the optimization - avoiding local optima
        #prior_init = [1.0, *([0.0]*channels) ]
    return prior_init

def argparser():
    """ Argument parser for the main script """
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # Model settings
    ms = parser.add_argument_group('Model settings')
    ms.add_argument('--model', type=str, default='vitae_ci', help='model to train')
    ms.add_argument('--ed_type', type=str, default='mlp,mlp', help='encoder/decoder type')
    ms.add_argument('--stn_type', type=str, default='gp_cpab', help='transformation type to use')
    ms.add_argument('--beta', type=float, default=1, help='beta value for beta-vae model') #16
    
    # Training settings
    ts = parser.add_argument_group('Training settings')
    ts.add_argument('--n_epochs', type=int, default=500, help='number of epochs of training')
    ts.add_argument('--eval_epoch', type=int, default=5, help='when to evaluate log(p(x))')
    ts.add_argument('--batch_size', type=int, default=2, help='size of the batches') # batch=10,warmup=10, epochs=100
    ts.add_argument('--warmup', type=int, default=5, help='number of warmup epochs for kl-terms')
    ts.add_argument('--lr', type=float, default=1e-3, help='learning rate for adam optimizer') # 1e-5 for 2 sequence deepseq 2 seqs other# 1e-3, 1e-7 # weird case for 550 iters and 1e-4, works in trans but regular in reconstruction
    
    # Paths to use
    paths = parser.add_argument_group('Paths')
    paths.add_argument('--path_orig', type=str, default="../data/WW10seeds_rawnoHoles.fasta", help='original sequence to deform')    #orig_3aa.fasta orig_3aag.fasta
    paths.add_argument('--path_aligned_orig', type=str, default="../data/WW10Seeds_100samplesMSA.fasta", help='original sequence to deform')    #orig_3aa.fasta orig_3aag.fasta
 
    paths.add_argument('--path_preexist_linear', type=str, default="../../models/CPABlinear3.pth", help='prebuilt model using linear case') 
    paths.add_argument('--path_preexist_gp', type=str, default="../../models/CPABGPB3.pth", help='prebuilt model using gp case')
    paths.add_argument('--path_automated_report', type=str, default="../../Results", help='path to save automatic report') 
    paths.add_argument('--logdir', type=str, default="../../Results", help='where to store results')

    gpsetup = parser.add_argument_group('GPSetup')
    #gpsetup.add_argument('--Task', type=int, default = 8, help='Amount of channels in multitask-gp estimator')
    #gpsetup.add_argument('--Initialization', type=list, default = [0.0, 0.143, 0.143, 0.143, 0.143, 0.143, 0.143, 0.143], help='multitaks-gp initialization') 
    gpsetup.add_argument('--Task', type=int, default = 22, help='Amount of channels in multitask-gp estimator')
    gpsetup.add_argument('--Initialization', type=list, default = define_prior_init(22, space='default'), help='multitaks-gp initialization') # before it was 19 and works good in current setup with uniform distributed values in prior
    #gpsetup.add_argument('--Task', type=int, default = 11, help='Amount of channels in multitask-gp estimator')
    #gpsetup.add_argument('--Initialization', type=list, default = [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], help='multitaks-gp initialization') #[0.0, 0.33, 0.33, 0.33] #[0.5, 0.5]
    gpsetup.add_argument('--Lengthscale', type=float, default = 0.5, help='lengthscale on square-exponential kernel') # *** 0.5 *** #0.25, #0.5 + init gives better for 2 gaps case, # 1.25
    gpsetup.add_argument('--noise_constraint', type=list, default = [0,0.0004], help='noise constrain') #16
    gpsetup.add_argument('--Option', type=str, default = 'multitask', help='noise constrain') #16

    # CPAB features
    cpab = parser.add_argument_group('CPAB')
    cpab.add_argument('--device', type=str, default="cpu", help='device')
    cpab.add_argument('--modeflag', type=str, default="1D", help='dimensionality of tesselation')
    cpab.add_argument('--window_grid', type=int, default=78-1, help='number of tesselation cells') #6 leave it as default in shape_input*2(corresponding to the length of seq)
    cpab.add_argument('--channels', type=int, default=78-1, help='amount of channels for estimation --deprecated')
    cpab.add_argument('--interpolation_type', type=str, default="GP", help='type of interpolation between maps') 

    # Hyper settings
    hp = parser.add_argument_group('Variational settings')
    hp.add_argument('--latent_dim', type=int, default=40, help='dimensionality of the latent space') #5, 10, 40 is the bst one for WW
    hp.add_argument('--density', type=str, default='softmax', help='output density')  # bernoulli  gaussian
    hp.add_argument('--eq_samples', type=int, default=1, help='number of MC samples over the expectation over E_q(z|x)')
    hp.add_argument('--iw_samples', type=int, default=1, help='number of importance weighted samples')
    
    # Parse and return
    args = parser.parse_args()
    return args

print(__package__)
args = argparser()
'''---------------------------------------------------------'''
std = configManager(args)

device = std.parserinfo('device')
modeflag = std.parserinfo('modeflag')
window_grid = std.parserinfo('window_grid')
channels = std.parserinfo('channels')
option = std.parserinfo('Option')
beta = std.parserinfo('beta')
#lossmetric = std.parserinfo('lossfunctmetric')

path = std.parserinfo('path_orig')
path_aligned = std.parserinfo('path_aligned_orig')
path_preexist_model = std.parserinfo('path_preexist_linear')
path_preexist_modelGP = std.parserinfo('path_preexist_gp')

#self.config, self.constrain, self.tasks, self.interpolation_type, self.option 
gp_params = std.get_config_vals(['noise_constraint','Task','interpolation_type','Option','Lengthscale','Initialization'])
print('Ok')
alphabets = ['?','-', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] #-> for 4 sequences


None
Ok


In [3]:
from src.gp_cpab.src.extra import experiment_utilities
from src.seqsDataLoader import seqsReader, seqsDatasetLoader, Sequence_Data_Loader

# Raw Sequences, to see if we can align the sequences somehow

c2i, i2c, i2i = seqsReader._predefine_encoding(alphabets)
dataset_msa = seqsDatasetLoader(pathBLAT_data = path, alphabet = alphabets, enable_variable_length=True, device=device)
#x1 = dataset_msa.prot_space
dataset_aligned_msa = seqsDatasetLoader(pathBLAT_data = path_aligned, alphabet = alphabets, enable_variable_length=True, device=device)

def recursive_shifting_grid(ts, cont, index, dT):
    if cont<index.shape[-1]:
        ts[:,index[cont]::]+=dT  #shift should be defined by the number of neighboors in the indexes (amount of gaps) times the dT, CORRECT!!!
        return recursive_shifting_grid(ts, cont+1,index,dT)

def find_best_alignment_deformation_target_from_ref(target, T):
    target_argmax_seqs = target_argmax_seqs = target.prot_space.argmax(-1)[0:10]
    num_seqs, len_seqs = target_argmax_seqs.shape
    Ugrid_system = T.uniform_meshgrid([len_seqs,len_seqs]).repeat(num_seqs,1,1)

    for cont, i in enumerate(target_argmax_seqs):
        seq_gaps_location_index = (i == 1).nonzero(as_tuple=True)[0]
        recursive_shifting_grid( Ugrid_system[cont], 0, seq_gaps_location_index, dT=1/(len_seqs-1))
    print('Ok')


/workspaces/Explicit_Disentanglement_Proteins/src/seqsDataLoader.py:165: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.prot_space = torch.tensor(self.prot_space, dtype=torch.float32, device=device)


In [4]:

window_grid= dataset_aligned_msa.prot_space.argmax(-1).shape[1]
ndim = [window_grid]
T = gp_cpab(ndim, std, backend='pytorch', device=device, zero_boundary=False,
                                          volume_perservation=False, override=False, argparser_gpdata = gp_params)

find_best_alignment_deformation_target_from_ref(dataset_aligned_msa, T)

theta_ref = torch.autograd.Variable(T.identity(1, epsilon=1e-6), requires_grad=True)
optimizer = torch.optim.AdamW([theta_ref], lr=1e-2)


Ok


In [6]:
print(__package__)
from tensorboardX import SummaryWriter
import time, os, datetime
# Logdir for results
'''
if args.logdir == '':
    logdir = args.logdir + '/res/' + 'gp_cpab_tests' + '/' + datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')
else:
    logdir = args.logdir + '/res/' + 'gp_cpab_tests'
  
writer = SummaryWriter(log_dir=logdir)
'''

def training_theta_optima(theta_est, maxiter, optimizer, T,x1, ref, inverse):

    pb = tqdm(desc='Alignment of samples', unit='iters', total=maxiter)
    kl_loss = torch.nn.GaussianNLLLoss()

    for i in range(maxiter):
        optimizer.zero_grad()

        if inverse == True:
            opt_est = -theta_est

        x1_trans = T.transform_grid(x1, opt_est).squeeze(1)
        loss = kl_loss(x1_trans,ref, var = torch.ones( x1_trans.shape ,requires_grad=True) )
            #loss = loss_function(method = lossmetric, input = x1_trans, target = ref_msa, forw_per=forw_per) # 'JSD','Soft_Label_KLD'

        loss.backward(retain_graph=True)
        optimizer.step()

        gc.collect()
        torch.cuda.empty_cache()
        #torch.save(theta_est, path_preexist_model)

        pb.update()
        #pb.set_postfix({'loss': str(loss.item())})
        pb.set_postfix({'loss': 'loss: ' + str(  loss.item()) + ',  value: ' + str(x1_trans)  })

        #msa_num.append(x1_trans.detach().numpy().squeeze())


    pb.close()
    #shapes = ref_msa.squeeze().shape
    #msa_num = np.vstack(msa_num).reshape(-1, shapes[0], shapes[1])


    return theta_est

None


In [8]:
sequence = 'ABCD'
ff= enumerate(sequence)

window_grid=10
ndim = [window_grid]
T = gp_cpab(ndim, std, backend='pytorch', device=device, zero_boundary=False,
                                          volume_perservation=False, override=False, argparser_gpdata = gp_params)

theta_ref = torch.autograd.Variable(T.identity(1, epsilon=1e-6), requires_grad=True)
optimizer = torch.optim.AdamW([theta_ref], lr=1e-2)

target = torch.tensor([0.02, 0.4, 0.66, 0.97, 1., 1.15, 1.20, 1.22, 1.25, 1.35], requires_grad=True).reshape(1,-1)
input = T.uniform_meshgrid([10,10]).squeeze(1)

Theta_optimal = training_theta_optima(theta_ref, 1500, optimizer, T, input, target, inverse=True)

T.transform_grid( input, -Theta_optimal)



print('Ok')

Alignment of samples:   0%|          | 1/1500 [00:00<03:22,  7.41iters/s, loss=loss: 1.0378837585449219,  value: tensor([[-8.7196e-07,  1.1111e-01,  2.2222e-01,  3.3333e-01,  4.4444e-01,
          5.5556e-01,  6.6667e-01,  7.7778e-01,  8.8889e-01,  1.0000e+00]],
Alignment of samples:   0%|          | 2/1500 [00:00<03:37,  6.90iters/s, loss=loss: 1.0378837585449219,  value: tensor([[-8.7196e-07,  1.1111e-01,  2.2222e-01,  3.3333e-01,  4.4444e-01,
          5.5556e-01,  6.6667e-01,  7.7778e-01,  8.8889e-01,  1.0000e+00]],
Alignment of samples:   0%|          | 2/1500 [00:00<03:37,  6.90iters/s, loss=loss: 0.9889078736305237,  value: tensor([[0.0087, 0.1201, 0.2315, 0.3435, 0.4553, 0.5672, 0.6793, 0.7914, 0.9034,
Alignment of samples:   0%|          | 3/1500 [00:00<03:32,  7.04iters/s, loss=loss: 0.9889078736305237,  value: tensor([[0.0087, 0.1201, 0.2315, 0.3435, 0.4553, 0.5672, 0.6793, 0.7914, 0.9034,
Alignment of samples:   0%|          | 3/1500 [00:00<03:32,  7.04iters/s, loss=loss: 0

Ok
